In [1]:
import numpy as np

In [2]:
import pandas as pd

df = pd.read_json('투자 유치 100억이상 회사 복지 혜택.json')

#pip install sklearn 혹은 conda install sklearn 을 통해 사이킷런(sklearn) 오픈 소스 다운로드

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import konlpy 

df = df.fillna('')

df_columns = df.columns.values

df_columns= np.delete(df_columns,[0])

# 자기계발 칼럼을 임시 리스트에 담아줌
# df_self_imporvement_list = df['self development'].tolist()

# TF - IDF 기법을 사용해서 텍스트의 등장 빈도를 체크해 저장
# tfidf_vect_simple = TfidfVectorizer()
# feature_vect_simple = tfidf_vect_simple.fit_transform(df_self_imporvement_list)

# 등장 빈도에 기반하여, 코사인 유사도 알고리즘 적용

# similarity_simple_pair = cosine_similarity(feature_vect_simple[2] , feature_vect_simple)
# result_list = similarity_simple_pair.tolist()[0]


# result_list를 df 우측에 삽입해서, 유사도가 높은 순으로 정렬
# df['self_development_result'] = result_list
# df.sort_values(by='self_development_result', ascending=False).head(100)


# df.sort_values(by='self_development_result').tail(20)  # ascending을 바꿔주거나
#.head(20)이 아닌 .tail(20)을 하면 됩니다

In [3]:
# def tfidf_vect_func(df, x, target_row, tokenizer):
    
#     tfidf_vect = TfidfVectorizer()
#     feature_vect = tfidf_vect.fit_transform(df[f'{x}'])
#     similarity_simple_pair = cosine_similarity(feature_vect[2], feature_vect)
#     result_list = similarity_simple_pair.tolist()[0]
#     df[f'{x}_result'] = result_list

In [4]:
def tfidf_vect_func(df, target_col, target_row, tokenizer):

    col_tokenized = [ tokenizer.morphs(item) for item in df[target_col] ] 
    col_tokenized = [ ' '.join(item) for item in col_tokenized ]

    tfidf_vect = TfidfVectorizer()
    feature_vect = tfidf_vect.fit_transform(col_tokenized)

    similarity_simple_pair = cosine_similarity(feature_vect[target_row], feature_vect)
    result_list = similarity_simple_pair.tolist()[0]
    df[f'{target_col}_result'] = result_list
    
    return df

In [5]:
# for row in df['transportaion']:
#     print(' ' in row)

In [6]:
from tkinter import *
from tkinter import ttk
from tkinter import messagebox

In [7]:
choice = input('회사를 입력하시려면 0, 복지를 입력하시려면 1 을 입력하세요. : ')

회사를 입력하시려면 0, 복지를 입력하시려면 1 을 입력하세요. : 0


In [8]:
a = df['Company_name'].values.tolist()

In [9]:
%%capture cap --no-stderr 
from tkinter import *
from tkinter import ttk
from tkinter import messagebox

#콤보박스 선택 값 돌려주기
def change1(index, value, op):
    print(brand_combo.get())
#     print(brand_combo.current())

    brand_index = brand_combo.current()
    label = ttk.Label(win, text=brand_index)
    label.grid(row = 0, column=1)

brand = a

# window 띄우기
win = Tk ()
win.title("스타트업 회사 선택")
win.geometry('400x200')
win.resizable(FALSE, FALSE)

# brand 콤보박스
str1 = StringVar()
str1.trace('w', change1)
brand_combo = ttk.Combobox(win, width=20, state='readonly', textvariable=str1, values=brand)
brand_combo.grid(row = 0, column=0)

win.bind('<Escape>', lambda e: win.destroy()) # ESC 버튼 누르면 창 닫기

win.mainloop()

In [10]:
global input_company

input_company = cap.stdout.strip() # cap 객체에 저장된 값을 test 변수에 할당
print(input_company)

모두싸인


In [12]:
df[df['Company_name'] == input_company]

,Company_name,Work type,"meals, snacks","annual leave, vacation","insurance, medical service",equipment,self development,transportaion
3,모두싸인,AM 10 - PM 07,"간식 제공, 석식 제공","장기근속휴가, 반반차, 반차, 월차","종합 검진 제공, 경조사비, 4대 보험","맥북 등 업무에 필요한 장비 제공, 듀얼 모니터","스터디 모임 지원, 세미나 참가비 지원, 원하는 도서 구입 가능, 컨퍼런스 참여 지원",원격근무 지원


In [13]:
print(input_company)

if choice == '0' : 
    print(input_company)
#     input_company = input("가장 마음에 드는 회사를 선택하세요. : ")
    target_row= df[df['Company_name'] == input_company].index[0]


    okt = konlpy.tag.Okt()

    for num, target_col in enumerate(df_columns):
        print('cosine_similarity 분석 중.. '+ str(num+1) + '/'+ str(len(df_columns)))
        tfidf_vect_func(df, target_col,target_row, okt)

    result_arr = []

    for column in df.columns.values :
        if 'result' in column:
            result_arr.append(column)


    df_result = df[result_arr]
    df_result_weight = df_result.copy()


    print()
    print ("근무 형태 : 0, 식사, 간식 : 1, 연차, 휴가 : 2 ,보험, 의료: 3 개인 장비: 5, 자기 계발: 5, 통근, 교통: 6")

    weight1 = input("가장 중요시하는 회사의 복지를 숫자의 형태로 입력해주세요. : ")  #  가장 중요하게 생각하는 회사의 복지
    weight2 = input("두 번쨰로 중요시하는 회사의 복지를 숫자의 형태로 입력해주세요. : ")
    weight3 = input("세 번쨰로 중요시하는 회사의 복지를 숫자의 형태로 입력해주세요. : ")

    df_result_weight.iloc[:,int(weight1)] = df_result_weight.iloc[:,int(weight1)] * 1.75
    df_result_weight.iloc[:,int(weight2)] = df_result_weight.iloc[:,int(weight1)] * 1.5
    df_result_weight.iloc[:,int(weight3)] = df_result_weight.iloc[:,int(weight1)] * 1.25

    df['sum'] = df_result_weight.cumsum(axis=1).iloc[:,-1]

    df_result_sort = df.sort_values(by='sum', ascending=False).copy()

    # print_df(df_result_sort.head(10))
    result_df = df_result_sort.head(10)

        
elif choice == '1' :
    
    df['mess'] = df.iloc[:,1:].sum(axis=1)
    df_columns = df.columns.values[1:]
    df = df.append(pd.Series(),ignore_index= True)
    last_index = df.iloc[[-1]].index[0]

    df.at[last_index,'mess'] =input('원하시는 복지를 입력해주세요.')



    okt = konlpy.tag.Okt()

    # for target_col in df_columns:
    #     tfidf_vect_func(df, target_col,target_row, okt)

    tfidf_vect_func(df,'mess',last_index,okt)
    df_result_sort = df.sort_values(by='mess_result', ascending=False).copy()
    df_result_sort_mess = df_result_sort.drop(columns='mess')

    # print_df(df_result_sort_mess.iloc[1:])
    result_df = df_result_sort_mess.iloc[1:]

else :
    # print('0 혹은 1 을 입력하세요.')
    result_df = '0 혹은 1 을 입력하세요.'
    

result_df['recruit'] = 'https://www.wanted.co.kr/search?query=' + result_df['Company_name']

def make_clickable(link):
    text = link.split('=')[1]
    return f'<a target="_blank" href="{link}">{text}</a>'

result_df = result_df.style.format({'recruit': make_clickable})    
    
result_df

모두싸인
모두싸인
cosine_similarity 분석 중.. 1/7
cosine_similarity 분석 중.. 2/7
cosine_similarity 분석 중.. 3/7
cosine_similarity 분석 중.. 4/7
cosine_similarity 분석 중.. 5/7
cosine_similarity 분석 중.. 6/7
cosine_similarity 분석 중.. 7/7

근무 형태 : 0, 식사, 간식 : 1, 연차, 휴가 : 2 ,보험, 의료: 3 개인 장비: 5, 자기 계발: 5, 통근, 교통: 6
가장 중요시하는 회사의 복지를 숫자의 형태로 입력해주세요. : 1
두 번쨰로 중요시하는 회사의 복지를 숫자의 형태로 입력해주세요. : 2
세 번쨰로 중요시하는 회사의 복지를 숫자의 형태로 입력해주세요. : 3


C:\Anaconda3\envs\kdigital\lib\site-packages\ipykernel_launcher.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Company_name,Work type,"meals, snacks","annual leave, vacation","insurance, medical service",equipment,self development,transportaion,Work type_result,"meals, snacks_result","annual leave, vacation_result","insurance, medical service_result",equipment_result,self development_result,transportaion_result,sum,recruit
3,모두싸인,AM 10 - PM 07,"간식 제공, 석식 제공","장기근속휴가, 반반차, 반차, 월차","종합 검진 제공, 경조사비, 4대 보험","맥북 등 업무에 필요한 장비 제공, 듀얼 모니터","스터디 모임 지원, 세미나 참가비 지원, 원하는 도서 구입 가능, 컨퍼런스 참여 지원",원격근무 지원,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,10.562500,모두싸인
102,포도트리,"유연 근무, AM 10 - PM 07","간식 무한 제공, 석식 제공, 중식 제공","기념일휴가 2일, 3년 근속시 3주, 출산 휴가",의료 실비 보험,"듀얼 모니터, iMac 27"", 맥북 프로 15""","동아리 지원, 세미나 참가비 지원",,0.797117,0.839637,0.072284,0.056219,0.179669,0.320132,0.000000,6.807037,포도트리
56,발란,"AM 10 - PM 07, 유연 근무","석식 제공, 고급 커피 제공, 간식 무한 제공","출산 휴가, 자율 휴가제, 월차","경조사비, 종합 검진 제공, 4대 보험","30"" 모니터, 맥북 프로 13"", 맥북 프로 15"", 듀얼 모니터",직무교육비지원,,0.797117,0.838925,0.422639,1.000000,0.210044,0.091020,0.000000,6.603624,발란
154,헬로마켓,"유연 근무, AM 10 - PM 07, AM 09 - PM 06","석식 제공, 고급 커피 제공, 간식 무한 제공",자율 휴가제,"경조사비, 4대 보험","맥북 프로 15""",,,0.715015,0.838925,0.151992,0.469957,0.130353,0.000000,0.000000,6.350812,헬로마켓
8,트릿지,,"고급 커피 제공, 간식 무한 제공, 석식 제공",월차,"종합 검진 제공, 4대 보험","듀얼 모니터, iMac 27"", 맥북 프로 15""",스터디 모임 지원,유류비 지원,0.000000,0.838925,0.497111,0.924017,0.179669,0.409699,0.168935,6.263746,트릿지
30,패스트파이브,AM 09:30 ~PM 6:30,"고급 커피 제공, 간식 무한 제공, 석식 제공","3년근속 시 10일 유급휴가, 연차 휴가 및 각종 경조사 지원, 생리 휴가, 출산 휴가","경조사비, 4대 보험",,"월별 도서구입비 5만원 지원, 세미나 참가비 지원, 동아리 지원, 패스트캠퍼스 강의 지원",야근시 식대 및 택시비 지원,0.170382,0.838925,0.101350,0.469957,0.000000,0.244081,0.139979,6.059886,패스트파이브
139,렌딧,"AM 10 - PM 07, 유연 근무","중식 제공, 고급 커피 제공, 석식 제공","한달에 한번 조기퇴근, 월차","종합 검진 제공, 경조사비, 4대 보험","iMac 27"", 맥북 프로 13"", 맥북 프로 15"", 듀얼 모니터",세미나 참가비 지원,,0.797117,0.715069,0.142330,1.000000,0.188356,0.352911,0.000000,6.031024,렌딧
48,플리토,AM 10 - PM 07,"석식 제공, 중식 제공",월차,4대 보험,,,,1.000000,0.753869,0.497111,0.273252,0.000000,0.000000,0.000000,5.947266,플리토
67,루닛,자율 출근,"간식 무한 제공, 고급 커피 제공, 커피 제공, 간식 제공, 석식 제공, 중식 제공","단축근로 모성보호, 육아휴직 제공, 출산 휴가, 자율 휴가제","경조사비, 종합 검진 제공, 4대 보험",원하는 장비 모두 제공,"건강지원비 연간 100만원, 도서구입비 지원, 세미나 참가비 지원",,0.000000,0.805858,0.054209,1.000000,0.332981,0.259801,0.000000,5.881225,루닛
61,잡플래닛,AM 10 - PM 07,"간식제공, 석식 제공, 점심식대지원","생일조기퇴근 및 선물, 매월마지막주 금요일 패밀리데이, 출산 휴가","경조사비, 의료실비지원, 4대 보험","맥북 프로 15"", 듀얼 모니터","업무관련도서구입, 자기계발비 지원, 세미나 참가비 지원",야근교통비지원,1.000000,0.627094,0.021992,0.137440,0.244788,0.314358,0.200491,5.874944,잡플래닛
